In [1]:
import pandas as pd

#Referência da base de dados utlizada: https://sci2s.ugr.es/keel/dataset/data/classification/thyroid-names.txt

dados = pd.read_csv('thyroid.csv')
dados.head()

,Age,Sex,On_thyroxine,Query_on_thyroxine,On_antithyroid_medication,Sick,Pregnant,Thyroid_surgery,I131_treatment,Query_hypothyroid,Query_hyperthyroid,Lithium,Goitre,Tumor,Hypopituitary,Psych,TSH,T3,TT4,T4U,FTI,Class
0,0.73,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00060,0.015,0.120,0.082,0.146,3
1,0.24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00025,0.030,0.143,0.133,0.108,3
2,0.47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00190,0.024,0.102,0.131,0.078,3
3,0.64,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00090,0.017,0.077,0.090,0.085,3
4,0.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00025,0.026,0.139,0.090,0.153,3


In [2]:
print(dados.columns)

Index(['Age', ' Sex', ' On_thyroxine', ' Query_on_thyroxine',
       ' On_antithyroid_medication', ' Sick', ' Pregnant', ' Thyroid_surgery',
       ' I131_treatment', ' Query_hypothyroid', ' Query_hyperthyroid',
       ' Lithium', ' Goitre', ' Tumor', ' Hypopituitary', ' Psych', ' TSH',
       ' T3', ' TT4', ' T4U', ' FTI', ' Class'],
      dtype='object')


In [3]:
x = dados[['Age', ' Sex', ' On_thyroxine', ' Query_on_thyroxine',
       ' On_antithyroid_medication', ' Sick', ' Pregnant', ' Thyroid_surgery',
       ' I131_treatment', ' Query_hypothyroid', ' Query_hyperthyroid',
       ' Lithium', ' Goitre', ' Tumor', ' Hypopituitary', ' Psych', ' TSH',
       ' T3', ' TT4', ' T4U', ' FTI',]]

In [4]:
x.head()

,Age,Sex,On_thyroxine,Query_on_thyroxine,On_antithyroid_medication,Sick,Pregnant,Thyroid_surgery,I131_treatment,Query_hypothyroid,Query_hyperthyroid,Lithium,Goitre,Tumor,Hypopituitary,Psych,TSH,T3,TT4,T4U,FTI
0,0.73,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00060,0.015,0.120,0.082,0.146
1,0.24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00025,0.030,0.143,0.133,0.108
2,0.47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00190,0.024,0.102,0.131,0.078
3,0.64,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00090,0.017,0.077,0.090,0.085
4,0.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00025,0.026,0.139,0.090,0.153


In [5]:
y = dados[' Class']

In [6]:
y.head()

0    3
1    3
2    3
3    3
4    3
Name:  Class, dtype: int64

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.2)

scaler = StandardScaler()
treino_x = scaler.fit_transform(raw_treino_x)
teste_x = scaler.fit_transform(raw_teste_x)

print(f'Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos')

Treinaremos com 5760 elementos e testaremos com 1440 elementos


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import Accuracy
from keras.utils import to_categorical

modelo = Sequential()

In [9]:
modelo.add(Dense(activation='relu', input_dim=len(x.columns), units=len(x.columns) , kernel_initializer = 'uniform'))
modelo.add(Dense(activation='relu', units=14))
modelo.add(Dense(activation='relu', units=14))
modelo.add(Dense(activation='relu', units=7))
modelo.add(Dense(activation='sigmoid', units=4))

In [10]:
modelo.compile(optimizer='adam', loss='mse', metrics=[Accuracy()])

In [11]:
modelo.save('mlp_model_exemple.h5')

In [12]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy = DummyClassifier()
dummy.fit(treino_x, treino_y)
previsoes_dummy = dummy.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes_dummy)
print(f'Taxa de acerto para o dummy classifier ou baseline foi {acuracia * 100:.2f}%')

Taxa de acerto para o dummy classifier ou baseline foi 84.72%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [13]:
modelo.fit(treino_x, to_categorical(treino_y), batch_size=24, epochs=500, validation_data=(teste_x, to_categorical(teste_y)))
previsoes = modelo.predict(teste_x)

Epoch 1/500
240/240 [==============================] - 1s 4ms/step - loss: 0.1675 - accuracy: 0.0000e+00 - val_loss: 0.0312 - val_accuracy: 1.7361e-04
Epoch 2/500
240/240 [==============================] - 0s 2ms/step - loss: 0.0257 - accuracy: 2.3485e-04 - val_loss: 0.0277 - val_accuracy: 1.7361e-04
Epoch 3/500
240/240 [==============================] - 0s 2ms/step - loss: 0.0229 - accuracy: 8.4759e-06 - val_loss: 0.0195 - val_accuracy: 1.7361e-04
Epoch 4/500
240/240 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 9.7171e-05 - val_loss: 0.0155 - val_accuracy: 1.7361e-04
Epoch 5/500
240/240 [==============================] - 0s 2ms/step - loss: 0.0131 - accuracy: 2.3988e-05 - val_loss: 0.0124 - val_accuracy: 1.7361e-04
Epoch 6/500
240/240 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 9.6346e-05 - val_loss: 0.0119 - val_accuracy: 1.7361e-04
Epoch 7/500
240/240 [==============================] - 0s 2ms/step - loss: 0.0092 - accuracy: 

In [14]:
from sklearn.metrics import classification_report

# normal (1) or suffers from hyperthyroidism (2) or hypothyroidism (3)

previsoes_argmax = np.argmax(previsoes, axis=1)
print(classification_report(teste_y, previsoes_argmax, target_names=["normal", 'hyperthyroidism', 'hypothyroidism']))

                 precision    recall  f1-score   support

         normal       0.87      0.87      0.87        31
hyperthyroidism       0.85      0.92      0.88        89
 hypothyroidism       0.99      0.99      0.99      1320

       accuracy                           0.98      1440
      macro avg       0.90      0.93      0.91      1440
   weighted avg       0.98      0.98      0.98      1440



In [15]:
from sklearn.metrics import accuracy_score


previsoes_argmax = np.argmax(previsoes, axis=1)
acuracia = accuracy_score(teste_y, previsoes_argmax) * 100
print(f'A acurácia foi {acuracia:.2f}%')

A acurácia foi 98.12%


In [16]:
from sklearn.metrics import confusion_matrix

previsoes_argmax = np.argmax(previsoes, axis=1)
matriz_confusao = confusion_matrix(teste_y, previsoes_argmax)
acuracia = ((matriz_confusao[0,0] + matriz_confusao[1,1] + matriz_confusao[2, 2]) / len(teste_y)) * 100
print(f'A acurácia foi {acuracia:.2f}%')


A acurácia foi 98.12%
